<a href="https://colab.research.google.com/github/Tomiwa-31/ai_crime_reporter_v1/blob/main/Ml_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate

In [ ]:
from google.colab import files
upload=files.upload()

Saving main_crime_alert.csv to main_crime_alert (2).csv


In [ ]:
import pandas as pd
from datasets import load_dataset,Dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
from sklearn.model_selection import train_test_split
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np


In [ ]:
df=pd.read_csv("main_crime_alert.csv")

In [ ]:
#x_train,x_test,y_train,y_test
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["text"], df["label"], test_size=0.3, random_state=42, stratify=df["label"]
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)


In [ ]:
#put the training text and label,the vaidation text and label ,test text and label all in a datframe
train_df=pd.DataFrame({"text":train_texts,"label":train_labels})
val_df=pd.DataFrame({"text":val_texts,"label":val_labels})
test_df=pd.DataFrame({"text":test_texts,"label":test_labels})

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
from datasets import DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# This is the final, organized dataset
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 154
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 33
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 33
    })
})


Train Teacher Model

In [ ]:
from datasets import ClassLabel, Value

# 1. Define the mapping between strings and integers
# The order is important: the first name gets ID 0, the second gets ID 1.
label_names = ["fake", "real"]

# 2. Cast the 'label' feature in your dataset to a ClassLabel
# This will automatically convert the strings to integers
def cast_labels(example):
    # This is a dummy function, the casting happens in the next step
    return example

# The actual casting is done by updating the dataset features
dataset_dict = dataset_dict.cast_column("label", ClassLabel(names=label_names))

# 3. (Optional but useful) Create mapping dictionaries for your model config
id2label = {i: name for i, name in enumerate(label_names)}
label2id = {name: i for i, name in enumerate(label_names)}

print("Label mapping:", id2label)

Casting the dataset:   0%|          | 0/154 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/33 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/33 [00:00<?, ? examples/s]

Label mapping: {0: 'fake', 1: 'real'}


In [ ]:
model_path="google-bert/bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_path)#select tokenizer for the bert model

id2label={0:"fake",1:"real"}#the output your model produces which is an integer, so this help help convert to text label
label2id={"fake":0,"real":1}#the input cus your model understands number:a dictionary that maps the string label to the integerid

#load model
model=AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=2,id2label=id2label,label2id=label2id)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#freeze the base model
for name,param in model.base_model.named_parameters():
  param.requires_grad=False

#unfreeze base model pooling layer
for name,param in model.named_parameters():
  if "pooler" in name:
    param.requires_grad = True

In [ ]:

# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

Preprocess text

In [ ]:
def preprocess(dataset):
  return tokenizer(dataset['text'],truncation=True)

In [ ]:
tokenize_data=dataset_dict.map(preprocess,batch_size=8)

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

In [ ]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
print("First tokenized example:", tokenize_data['train'][0])
print("Type of input_ids:", type(tokenize_data['train'][0]['input_ids']))
#print("Shape of input_ids (if tensor):", tokenize_data['train'][0]['input_ids'].shape)

First tokenized example: {'text': 'two cult groups clashed at a campus in port harcourt leaving 1 dead and 5 injured', 'label': 1, '__index_level_0__': 142, 'input_ids': [101, 2048, 8754, 2967, 22600, 2012, 1037, 3721, 1999, 3417, 22714, 2975, 1015, 2757, 1998, 1019, 5229, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Type of input_ids: <class 'list'>


In [ ]:
# This should print integers, not strings
print("Labels in training set:", set(dataset_dict['train']['label']))
# Should be {0, 1} not {'fake', 'real'}

Labels in training set: {0, 1}


Evaluation

In [ ]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

Train model

In [ ]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert_crime_alert_classifier",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenize_data["train"],
    eval_dataset=tokenize_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-2905571652.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.433800,0.201245,1.000000,1.000000
2,0.212000,0.059527,1.000000,1.000000
3,0.094300,0.029489,1.000000,1.000000
4,0.050300,0.008007,1.000000,1.000000
5,0.030400,0.004728,1.000000,1.000000
6,0.034000,0.004137,1.000000,1.000000
7,0.020600,0.003743,1.000000,1.000000
8,0.021900,0.002808,1.000000,1.000000
9,0.014400,0.002568,1.000000,1.000000
10,0.021600,0.002325,1.000000,1.000000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

TrainOutput(global_step=200, training_loss=0.09334629967808723, metrics={'train_runtime': 250.0455, 'train_samples_per_second': 6.159, 'train_steps_per_second': 0.8, 'total_flos': 15835996644480.0, 'train_loss': 0.09334629967808723, 'epoch': 10.0})

Tokens: ['what', 'is', 'the', 'capital', 'of', 'france', '?']
Input IDs: tensor([[ 101, 2054, 2003, 1996, 3007, 1997, 2605, 1029,  102]])


In [ ]:
# After trainer.train() is finished

# Create a directory to save your model
#save_directory = "./my_bert_crime_model"

# Save the trained model
#trainer.save_model(save_directory)

# Save the tokenizer (CRUCIAL step!)
#tokenizer.save_pretrained(save_directory)

#print(f"Model and tokenizer saved to {save_directory}")

Compress and download with code

In [ ]:
#from google.colab import files
#import shutil

# Compress the model directory into a zip file
#shutil.make_archive("my_bert_crime_model", 'zip', "my_bert_crime_model")

# Download the zip file
#files.download("my_bert_crime_model.zip")

save to google drive

In [ ]:
#from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

# Save directly to your Drive
#save_directory = "/content/drive/MyDrive/my_bert_crime_model"
#trainer.save_model(save_directory)
#tokenizer.save_pretrained(save_directory)

In [ ]:
# Install huggingface_hub if not already installed
!pip install huggingface_hub



In [ ]:
from huggingface_hub import notebook_login,logout,delete_repo

# This will open a widget for you to login
#logout()
#notebook_login()

In [ ]:
from huggingface_hub import whoami

# Check which user you're currently logged in as
user_info = whoami()
print(f"Logged in as: {user_info['name']}")

Logged in as: toladimeji


In [ ]:
# push model to hub
#trainer.push_to_hub()

In [ ]:
#Delete the model repository
#repo_id = "toladimeji/bert-phising-classifier_teacher"  # Replace with your actual model name

#try:
    #delete_repo(repo_id=repo_id, repo_type="model")
    #print(f"Successfully deleted model: {repo_id}")
#except Exception as e:
    #print(f"Error deleting model: {e}")